In [ ]:
import datetime
import os
import pickle
import pprint
import time
from abc import abstractmethod, ABC
from collections import defaultdict
from dataclasses import dataclass
from typing import Any, Dict
from typing import List

import lightgbm as lgb
import numpy as np
import optuna
import pandas as pd
import xgboost as xgb
from dotenv import load_dotenv
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
# from loguru import logger
from optuna.samplers import TPESampler
from optuna.trial import FrozenTrial
from scipy.stats import randint, uniform
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    f1_score,
    roc_auc_score,
    accuracy_score,
    roc_curve,
    log_loss,
    matthews_corrcoef,
    precision_score,
    recall_score,
    average_precision_score
)
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils.class_weight import compute_sample_weight

load_dotenv()


@dataclass
class Bundle:
    X: pd.DataFrame
    y: pd.Series
    ids: pd.DataFrame
    positives: int


@dataclass
class ModelResult:
    model_info: Dict[str, Any]
    result_df: pd.DataFrame
    hyper_params: Dict[str, Any]
    pipelines: List[Pipeline]

    def metrics(self) -> Dict[str, float]:
        return self.model_info.get("additional", {}).get("metrics", {})

    def get_trials(self) -> int:
        return self.model_info.get("additional", {}).get("study_trials", -1)

    def set_trials(self, n_trails: int):
        self.model_info.setdefault("additional", {})["study_trials"] = n_trails


class ModelTrainer(ABC):
    @abstractmethod
    def evaluate(
            self,
            bundle: Bundle,
            hyper_params: Dict[str, Any],
            n_splits: int,
            compute_shap: bool = False,
    ) -> ModelResult:
        raise NotImplementedError()


base_path = os.environ.get('BASE_PATH', '/kaggle/input/icr-identify-age-related-conditions')

train = pd.read_csv(f'{base_path}/train.csv')
test = pd.read_csv(f'{base_path}/test.csv')
greeks = pd.read_csv(f'{base_path}/greeks.csv')
# sample_submission = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')

num_cols = test.select_dtypes(include=['float64']).columns.tolist()
cat_cols = test.select_dtypes(include=['object']).columns.tolist()
cat_cols.remove('Id')

TARGET_COL = "Class"


def weighted_log_loss(y_true, y_pred) -> float:
    return log_loss(
        y_true, y_pred, sample_weight=compute_sample_weight("balanced", y_true)
    )


def weighted_matthews_corrcoef(y_true, y_pred) -> float:
    return matthews_corrcoef(
        y_true, y_pred, sample_weight=compute_sample_weight("balanced", y_true)
    )


def balanced_precision_recall(y_true, y_pred) -> float:
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return ((precision + recall) - abs(precision - recall)) / 2


def balanced_weighted_log_loss(y_true, y_pred, eps=1e-15):
    # Compute class prevalences
    prevalence_0 = np.mean(1 - y_true)
    prevalence_1 = np.mean(y_true)

    # Compute weights
    weight_0 = 1 / prevalence_0
    weight_1 = 1 / prevalence_1

    # Cap predicted probabilities
    y_pred = np.clip(y_pred, eps, 1 - eps)

    # Compute log loss for each class
    # np.mean() function is taking care of dividing by N0 and N1
    log_loss_0 = -np.mean((1 - y_true) * np.log(1 - y_pred))
    log_loss_1 = -np.mean(y_true * np.log(y_pred))

    # Combine the log losses
    total_log_loss = weight_0 * log_loss_0 + weight_1 * log_loss_1

    return float(total_log_loss / (weight_0 + weight_1))


def youdens_index(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    youdens_index = tpr - fpr
    return float(thresholds[np.argmax(youdens_index)])


def max_roc_distance(y_true, y_score):
    # maximizes the distance to the line of no-discrimination (diagonal line) in the ROC space.
    # Calculation maximizes the geometric mean of sensitivity and specificity.
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    gmeans = np.sqrt(tpr * (1 - fpr))
    ix = np.argmax(gmeans)
    return float(thresholds[ix])


SCORERS = {
    "roc_auc_score": (roc_auc_score, True),
    "recall_score": (recall_score, False),
    "precision_score": (precision_score, False),
    "f1_score": (f1_score, False),
    "log_loss": (log_loss, True),
    "weighted_log_loss": (weighted_log_loss, True),
    "balanced_weighted_log_loss": (balanced_weighted_log_loss, True),
    "accuracy_score": (accuracy_score, False),
    "matthews_corrcoef": (weighted_matthews_corrcoef, False),
    "balanced_precision_recall": (balanced_precision_recall, False),
    "average_precision_score": (average_precision_score, True),
}

OPTIMIZATION_OBJECTIVES = [
    ("minimize", "balanced_weighted_log_loss"),
    # ("minimize", "log_loss"),
    # ("maximize", "balanced_precision_recall"),
]

SCORING = OPTIMIZATION_OBJECTIVES[0][1]

MODEL_TYPE = "xgboost"

TAKE_TOP_FEATURES = 10

VERSION = f"{datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"


def build_feature_transformer() -> ColumnTransformer:
    numerical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler()),
        ]
    )
    # categorical_transformer = Pipeline(
    #     steps=[
    #         ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    #     ]
    # )
    if MODEL_TYPE == "xgboost":
        categorical_transformer = Pipeline(
            steps=[
                ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
                ("onehot", OneHotEncoder(handle_unknown="ignore")),
            ]
        )
    elif MODEL_TYPE == "lightgbm":
        categorical_transformer = Pipeline(
            steps=[
                ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
            ]
        )
    else:
        raise ValueError(f"{MODEL_TYPE} must be xgboost or lightgbm")

    return ColumnTransformer(
        transformers=[
            ("num", numerical_transformer, num_cols),
            ("cat", categorical_transformer, cat_cols),
        ]
    )


def build_pipeline(params: Dict[str, Any] | None = None, scale_pos_weight: float = 1.0) -> Pipeline:
    if MODEL_TYPE == "xgboost":
        if params:
            #
            clf = xgb.XGBClassifier(
                objective="binary:logistic",
                tree_method="hist",
                enable_categorical=True,
                # scale_pos_weight=scale_pos_weight,
                **params,
            )
        else:
            clf = xgb.XGBClassifier(
                objective="binary:logistic", tree_method="hist", enable_categorical=True,
                # scale_pos_weight=scale_pos_weight,
            )
    elif MODEL_TYPE == "lightgbm":
        clf = lgb.LGBMClassifier()
        if params:
            clf.set_params(**params)
    else:
        raise NotImplementedError(f"unknown model type {MODEL_TYPE}")

    pipeline = ImbPipeline(
        [
            (
                "preprocessor",
                build_feature_transformer(),
            ),
            ("smote", SMOTE(random_state=42)),
            ("classifier", clf),
        ]
    )
    return pipeline


def prepare_eval_X(X_train, X_test):
    eval_feature_transformer = build_feature_transformer()
    eval_feature_transformer.fit(X_train, X_test)
    X_eval = eval_feature_transformer.transform(X_test)
    return X_eval


class GradientBoosterTrainer(ModelTrainer):
    def train(self, params: Dict[str, Any], X, y) -> Pipeline:
        pipeline = build_pipeline(params=params)
        pipeline.fit(
            X,
            y
        )
        return pipeline

    def evaluate(
            self,
            bundle: Bundle,
            hyper_params: Dict[str, Any],
            n_splits: int,
            compute_shap: bool = False,
    ) -> ModelResult:
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        metrics = defaultdict(list)
        X = bundle.X
        y = bundle.y
        ids_list = []
        predictions = []
        print(f"Starting evaluation on {n_splits} splits")
        negative = int((bundle.y == 0).sum())
        positive = int((bundle.y == 1).sum())
        scale_pos_weight = negative / positive
        pipelines = []
        for train_index, test_index in cv.split(X, y):
            print(f"Fitting model on split: {len(ids_list)}")
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            ids_test = bundle.ids.values[test_index]
            pipeline = build_pipeline(params=hyper_params, scale_pos_weight=scale_pos_weight)
            pipeline.fit(
                X_train,
                y_train,
                classifier__early_stopping_rounds=20,
                classifier__eval_metric=["logloss"],
                classifier__eval_set=[(prepare_eval_X(X_train, X_test), y_test)],
                classifier__verbose=False,
            )
            best_iteration = pipeline['classifier'].best_iteration
            best_score = pipeline['classifier'].best_score
            print(f'best iter {best_iteration}. best score {best_score}')
            pipelines.append(pipeline)
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
            predictions.extend(y_pred_proba)
            y_pred = y_pred_proba > 0.5

            for metric_name, (fscore, needs_proba) in SCORERS.items():
                metrics[metric_name].append(
                    fscore(y_test, y_pred_proba if needs_proba else y_pred)
                )

            ids_list.append(ids_test)

        result_df = pd.DataFrame(
            bundle.ids, columns=["Id", "Class"]
        )
        # '', ''
        result_df["class_0"] = predictions
        result_df["class_1"] = [1 - prob for prob in predictions]
        # result_df["status"] = []

        metrics = {k: np.mean(v) for k, v in metrics.items()}

        additional = {
            "model_type": MODEL_TYPE,
            "metrics": metrics,
            "optimization_objectives": [obj for _, obj in OPTIMIZATION_OBJECTIVES],
            "hyper_prams": hyper_params.copy(),
            "youden_index": youdens_index(y, predictions),
            "max_gmeans_roc": max_roc_distance(y, predictions),
        }

        info_record = {
            k: v
            for k, v in metrics.items()
            if k
               in {
                   "balanced_weighted_log_loss",
                   "recall_score",
                   "precision_score",
                   "roc_auc_score",
                   "accuracy_score",
                   "f1_score"
               }
        }

        info_record["class_0"] = negative
        info_record["class_1"] = positive
        info_record["scale_pos_weight"] = scale_pos_weight
        print(f"Negative/Positive: {scale_pos_weight}")
        print(f"Calculated model metrics:\n{pprint.pformat(metrics)}")

        info_record["additional"] = additional
        info_record["model_version"] = VERSION

        # if compute_shap:
        #     info_record["features"] = json.dumps(model_shap)
        #     features_df = pd.concat(customers_shap)
        #     result_df = result_df.merge(features_df, on=["customer_id"])

        return ModelResult(
            model_info=info_record, result_df=result_df, hyper_params=hyper_params, pipelines=pipelines
        )


def preprocess_input(df: pd.DataFrame) -> Bundle:
    # df["label"] = [1 if l == "churned" else 0 for l in df.label.values]
    # positives = len(df[df.label == 1].index)
    # effective_date = df.effective_date.values[0]
    drop_cols = [
        col
        for col in df.columns
        if col not in cat_cols + num_cols
    ]
    X = df.drop(columns=drop_cols)
    print("X features:")
    print(X.columns)
    return Bundle(
        X=X,
        y=df[TARGET_COL],
        ids=df["Id"],
        positives=len(df[df[TARGET_COL] == 1].index)
    )


class HyperparametersOptimizer:
    def __init__(self, model: GradientBoosterTrainer, scaling_factor: float = 5000):
        self.model: ModelTrainer = model
        self.scaling_factor = scaling_factor

    def objective(self, trial, bundle: Bundle, n_splits: int, objectives: List[str]):
        if MODEL_TYPE == "xgboost":
            param = {
                "n_estimators": trial.suggest_int("n_estimators", 75, 200),
                "max_depth": trial.suggest_int("max_depth", 5, 15),
                "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
                "subsample": trial.suggest_float("subsample", 0.2, 0.8),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 0.8),
                "min_child_weight": trial.suggest_int("min_child_weight", 1, 30),
                "gamma": trial.suggest_float("gamma", 0, 1),
                "reg_alpha": trial.suggest_float("reg_alpha", 1e-7, 1e-2),
                "reg_lambda": trial.suggest_float("reg_lambda", 1e-7, 1e-2),
            }

        elif MODEL_TYPE == "lightgbm":
            param = {
                "classifier__n_estimators": trial.suggest_int("n_estimators", 35, 150),
                "classifier__max_depth": trial.suggest_int("max_depth", 5, 12),
                "classifier__learning_rate": trial.suggest_uniform(
                    "learning_rate", 0.001, 0.1
                ),
                "classifier__subsample": trial.suggest_uniform("subsample", 0.2, 0.8),
                "classifier__colsample_bytree": trial.suggest_uniform(
                    "colsample_bytree", 0.2, 0.9
                ),
                "classifier__min_child_weight": trial.suggest_int(
                    "min_child_weight", 1, 7
                ),
                "classifier__num_leaves": trial.suggest_int("num_leaves", 20, 100),
                "classifier__reg_alpha": trial.suggest_uniform("reg_alpha", 0, 1),
                "classifier__reg_lambda": trial.suggest_uniform("reg_lambda", 0, 1),
                "classifier__feature_fraction": trial.suggest_uniform(
                    "feature_fraction", 0.4, 1.0
                ),
                "classifier__bagging_fraction": trial.suggest_uniform(
                    "bagging_fraction", 0.4, 1.0
                ),
                "classifier__min_child_samples": trial.suggest_int(
                    "min_child_samples", 5, 100
                ),
            }

        else:
            raise ValueError("Invalid model_type. Must be 'xgboost' or 'lightgbm'.")

        # Perform cross-validation and take the mean of the scores
        result = self.model.evaluate(
            bundle=bundle, hyper_params=param, n_splits=n_splits, compute_shap=False
        )
        metrics = result.metrics()
        scores = [metrics[objective] for objective in objectives]
        print(f"Trial {trial.number} CV {objectives}: {scores}")
        return tuple(scores)

    def optimize_params(
            self, bundle: Bundle, n_splits: int, n_trials=500
    ) -> Dict[str, Any]:
        sampler = TPESampler(seed=42)
        print(f"Objectives {OPTIMIZATION_OBJECTIVES}")
        directions, objectives = zip(*OPTIMIZATION_OBJECTIVES)
        study = optuna.create_study(
            directions=directions,
            sampler=sampler,
            pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
            study_name=f"icr-conditions-{VERSION}",
        )
        print(
            f"Starting hyperparameters search. {n_trials} trials, {n_splits} cv splits"
        )
        start = time.time()
        study.optimize(
            lambda trial: self.objective(trial, bundle, n_splits, objectives),
            n_trials=n_trials,
            show_progress_bar=True,
        )
        print(f"Pareto Front")

        def aggregate_metrics(x: FrozenTrial):
            print(f"Trial {x.number} - values: {x.values}")
            return sum(
                [
                    v if directions[i] == "maximize" else -v
                    for i, v in enumerate(x.values)
                ]
            )

        best_trial = sorted(study.best_trials, key=lambda x: aggregate_metrics(x))[-1]

        print(
            f"Hyperparameters search took {(time.time() - start) / 60} minutes."
        )
        print(f"Best values: {pprint.pformat(best_trial.values)}")
        print(f"Best hyper-parameters: {pprint.pformat(best_trial.params)}")
        return best_trial.params


def find_by_randomized_cv_search(
        X_train,
        y_train,
        pipeline,
        scoring: str = "roc_auc",
        model_type: str = "xgboost",
        n_iter=500,
):
    if model_type == "xgboost":
        param_dist = {
            "classifier__n_estimators": randint(35, 200),
            "classifier__max_depth": randint(5, 12),
            "classifier__learning_rate": uniform(0.001, 0.1),
            "classifier__subsample": uniform(0.2, 0.8),
            "classifier__colsample_bytree": uniform(0.2, 0.8),
            "classifier__min_child_weight": randint(1, 30),
            "classifier__gamma": uniform(0, 1),
        }
    elif model_type == "lightgbm":
        param_dist = {
            "classifier__n_estimators": randint(50, 300),
            "classifier__max_depth": randint(3, 10),
            "classifier__learning_rate": uniform(0.01, 0.3),
            "classifier__subsample": uniform(0.5, 0.5),
            "classifier__colsample_bytree": uniform(0.5, 0.5),
            "classifier__min_child_weight": randint(1, 7),
            "classifier__num_leaves": randint(20, 100),
            "classifier__reg_alpha": uniform(0, 1),
            "classifier__reg_lambda": uniform(0, 1),
        }
    else:
        raise ValueError("Invalid model_type. Must be 'xgboost' or 'lightgbm'.")

    search = RandomizedSearchCV(
        pipeline,
        param_distributions=param_dist,
        n_iter=n_iter,
        scoring=scoring,
        cv=5,
        n_jobs=-1,
        random_state=42,
        verbose=1,
    )

    search.fit(X_train, y_train)
    best_params = search.best_params_
    best_params = {
        key.replace("classifier__", ""): value for key, value in best_params.items()
    }
    print(f"Best parameters: {pprint.pformat(best_params)}")
    return best_params


def run_main(
        hp_trials: int = 500,
) -> ModelResult:
    model = GradientBoosterTrainer()
    optimizer = HyperparametersOptimizer(model=model)
    start = time.time()

    print(f"Loading features and labels version {VERSION}")

    bundle = preprocess_input(train)
    
    
    hyper_params = {'colsample_bytree': 0.33052237200491763,
                                'gamma': 0.44216411057904137,
                                'learning_rate': 0.0869668401043195,
                                'max_depth': 9,
                                'min_child_weight': 3,
                                'n_estimators': 196,
                                'reg_alpha': 0.0042716453721107425,
                                'reg_lambda': 0.006124657768172271,
                                'subsample': 0.6486718367208028}
    
#     hyper_params = optimizer.optimize_params(
#         bundle,
#         n_splits=4,
#         n_trials=hp_trials
#     )
    print(
        f"Training, Predicting"
    )
    # test_splits = round(bundle.positives / 25)
    test_splits = 5
    model_result = model.evaluate(
        bundle=bundle,
        n_splits=test_splits,
        hyper_params=hyper_params,
        # compute_shap=True,
    )
    print(f'Model {VERSION} Info:')
    print(pprint.pformat(model_result.model_info))
    results_path = os.environ.get('RESULTS_PATH', None)
    if results_path:
        res_path = f'{results_path}/{VERSION}'
        os.mkdir(res_path)
        pd.DataFrame.from_records([model_result.model_info]).to_csv(f'{res_path}/model_info.csv')
        model_result.result_df.to_csv(f'{res_path}/results.csv')
        with open(f'{res_path}/pipeline.pkl', 'wb') as f:
            pickle.dump(model_result.pipelines, f)
        print(f"Model for {VERSION} saved")
        
    test_features = test.drop(columns=['Id'])
    test_predictions = []
    for model in model_result.pipelines:
        proba = model.predict_proba(test_features)  # picking the probabilities for both classes
        test_predictions.append(proba)

    # convert list of arrays to 3D array and then take mean along axis 0
    test_predictions = np.mean(np.array(test_predictions), axis=0)

    # Create a submission dataframe and save it to a csv file
    submission = pd.DataFrame(test_predictions, columns=['class_0', 'class_1'])
    submission.insert(0, 'Id', test['Id'])
    submission.to_csv('submission.csv', index=False)

    print(f"ICR Conditions Model Run Completed. Overall Execution Took: {(time.time() - start) / 60} minutes")

run_main(hp_trials=1)
